In [ ]:
# Imports
import sys
import json
import warnings
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.ensemble import HistGradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_validate, learning_curve, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Add project root
sys.path.insert(0, str(Path.cwd().parent))

from ml.preprocessing import (
    create_preprocessing_pipeline,
    prepare_data,
    get_numeric_features,
    get_categorical_features
)

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

# Paths
DATA_PATH = Path('../data/train.csv')
MODELS_DIR = Path('../ml/models')
FIGURES_DIR = Path('../report/figures')

MODELS_DIR.mkdir(parents=True, exist_ok=True)
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

print("Setup complete!")

## 1. Load and Prepare Data

In [ ]:
# Load data
df = pd.read_csv(DATA_PATH)
print(f"Dataset shape: {df.shape}")

# Prepare features and target
X, y = prepare_data(df, target_col='SalePrice', log_transform_target=True)

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Target (log1p) - Min: {y.min():.2f}, Max: {y.max():.2f}, Mean: {y.mean():.2f}")

## 2. Define Models and Preprocessing

In [ ]:
# Create preprocessing pipeline
preprocessor = create_preprocessing_pipeline(scale_numeric=True)

# Define 3 models
models = {
    'Ridge': Ridge(alpha=10.0),
    'RandomForest': RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    ),
    'HistGradientBoosting': HistGradientBoostingRegressor(
        max_iter=200,
        max_depth=10,
        learning_rate=0.1,
        min_samples_leaf=20,
        random_state=42
    )
}

print(f"Models to compare: {list(models.keys())}")

## 3. 5-Fold Cross-Validation

In [ ]:
# Evaluate each model with 5-fold CV
results = {}
pipelines = {}

scoring = {
    'neg_mse': 'neg_mean_squared_error',
    'neg_mae': 'neg_mean_absolute_error',
    'r2': 'r2'
}

for name, model in models.items():
    print(f"\nTraining: {name}")
    
    # Create full pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    pipelines[name] = pipeline
    
    # 5-Fold CV
    cv_results = cross_validate(
        pipeline, X, y,
        cv=5,
        scoring=scoring,
        return_train_score=True,
        n_jobs=-1
    )
    
    # Calculate metrics
    rmse = np.sqrt(-cv_results['test_neg_mse'])
    mae = -cv_results['test_neg_mae']
    r2 = cv_results['test_r2']
    
    results[name] = {
        'rmse_mean': rmse.mean(),
        'rmse_std': rmse.std(),
        'mae_mean': mae.mean(),
        'mae_std': mae.std(),
        'r2_mean': r2.mean(),
        'r2_std': r2.std()
    }
    
    print(f"  RMSE: {rmse.mean():.4f} ± {rmse.std():.4f}")
    print(f"  MAE:  {mae.mean():.4f} ± {mae.std():.4f}")
    print(f"  R²:   {r2.mean():.4f} ± {r2.std():.4f}")

In [ ]:
# Results summary table
results_df = pd.DataFrame(results).T
results_df = results_df.round(4)
results_df['RMSE'] = results_df['rmse_mean'].astype(str) + ' ± ' + results_df['rmse_std'].astype(str)
results_df['MAE'] = results_df['mae_mean'].astype(str) + ' ± ' + results_df['mae_std'].astype(str)
results_df['R²'] = results_df['r2_mean'].astype(str) + ' ± ' + results_df['r2_std'].astype(str)

print("\n" + "="*60)
print("5-FOLD CROSS-VALIDATION RESULTS")
print("="*60)
results_df[['RMSE', 'MAE', 'R²']]

## 4. CV Comparison Plot

In [ ]:
# Plot CV comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

model_names = list(results.keys())
x = np.arange(len(model_names))
width = 0.6
colors = ['#3498db', '#2ecc71', '#e74c3c']

# RMSE
ax = axes[0]
rmse_means = [results[m]['rmse_mean'] for m in model_names]
rmse_stds = [results[m]['rmse_std'] for m in model_names]
bars = ax.bar(x, rmse_means, width, yerr=rmse_stds, color=colors, capsize=5, alpha=0.8)
ax.set_ylabel('RMSE (log scale)', fontsize=11)
ax.set_title('RMSE Comparison (5-Fold CV)', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(model_names, fontsize=10)
for bar, val in zip(bars, rmse_means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
            f'{val:.4f}', ha='center', va='bottom', fontsize=9)

# MAE
ax = axes[1]
mae_means = [results[m]['mae_mean'] for m in model_names]
mae_stds = [results[m]['mae_std'] for m in model_names]
bars = ax.bar(x, mae_means, width, yerr=mae_stds, color=colors, capsize=5, alpha=0.8)
ax.set_ylabel('MAE (log scale)', fontsize=11)
ax.set_title('MAE Comparison (5-Fold CV)', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(model_names, fontsize=10)
for bar, val in zip(bars, mae_means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.003,
            f'{val:.4f}', ha='center', va='bottom', fontsize=9)

# R2
ax = axes[2]
r2_means = [results[m]['r2_mean'] for m in model_names]
r2_stds = [results[m]['r2_std'] for m in model_names]
bars = ax.bar(x, r2_means, width, yerr=r2_stds, color=colors, capsize=5, alpha=0.8)
ax.set_ylabel('R² Score', fontsize=11)
ax.set_title('R² Comparison (5-Fold CV)', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(model_names, fontsize=10)
ax.set_ylim(0, 1)
for bar, val in zip(bars, r2_means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{val:.4f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'cv_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'cv_comparison.png'}")

## 5. Select Best Model

In [ ]:
# Select best model (lowest RMSE)
best_model_name = min(results, key=lambda x: results[x]['rmse_mean'])
best_pipeline = pipelines[best_model_name]
best_results = results[best_model_name]

print("=" * 50)
print(f"BEST MODEL: {best_model_name}")
print("=" * 50)
print(f"RMSE: {best_results['rmse_mean']:.4f} ± {best_results['rmse_std']:.4f}")
print(f"MAE:  {best_results['mae_mean']:.4f} ± {best_results['mae_std']:.4f}")
print(f"R²:   {best_results['r2_mean']:.4f} ± {best_results['r2_std']:.4f}")

## 6. Train Best Model and Generate Predictions

In [ ]:
# Fit best model on full data
best_pipeline.fit(X, y)
y_pred = best_pipeline.predict(X)

# Also get cross-validated predictions for unbiased evaluation
y_pred_cv = cross_val_predict(best_pipeline, X, y, cv=5)

print(f"Training predictions: {len(y_pred)}")
print(f"CV predictions: {len(y_pred_cv)}")

## 7. Residual Analysis

In [ ]:
# Residuals using CV predictions (unbiased)
residuals = y.values - y_pred_cv

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Residuals vs Predicted
ax = axes[0]
ax.scatter(y_pred_cv, residuals, alpha=0.5, s=30, c='#3498db')
ax.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax.set_xlabel('Predicted Values (log scale)', fontsize=11)
ax.set_ylabel('Residuals', fontsize=11)
ax.set_title(f'{best_model_name}: Residuals vs Predicted (CV)', fontsize=12, fontweight='bold')

# Residual distribution
ax = axes[1]
ax.hist(residuals, bins=50, edgecolor='white', color='#2ecc71', alpha=0.7)
ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero')
ax.axvline(x=residuals.mean(), color='blue', linestyle='--', linewidth=2, 
           label=f'Mean: {residuals.mean():.4f}')
ax.set_xlabel('Residuals', fontsize=11)
ax.set_ylabel('Frequency', fontsize=11)
ax.set_title(f'{best_model_name}: Residual Distribution', fontsize=12, fontweight='bold')
ax.legend()

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'residuals.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'residuals.png'}")

## 8. Predicted vs Actual Plot

In [ ]:
# Predicted vs Actual using CV predictions
fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(y.values, y_pred_cv, alpha=0.5, s=30, c='#3498db')

# Perfect prediction line
min_val = min(y.min(), y_pred_cv.min())
max_val = max(y.max(), y_pred_cv.max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')

# Metrics
rmse = np.sqrt(mean_squared_error(y, y_pred_cv))
r2 = r2_score(y, y_pred_cv)

ax.set_xlabel('Actual Values (log scale)', fontsize=11)
ax.set_ylabel('Predicted Values (log scale)', fontsize=11)
ax.set_title(f'{best_model_name}: Predicted vs Actual (CV)\nRMSE={rmse:.4f}, R²={r2:.4f}', 
             fontsize=12, fontweight='bold')
ax.legend(loc='upper left')
ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'predicted_vs_actual.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'predicted_vs_actual.png'}")

## 9. Learning Curve

In [ ]:
# Learning curve
train_sizes = np.linspace(0.1, 1.0, 10)

train_sizes_abs, train_scores, test_scores = learning_curve(
    best_pipeline, X, y,
    train_sizes=train_sizes,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Convert to RMSE
train_rmse = np.sqrt(-train_scores)
test_rmse = np.sqrt(-test_scores)

train_mean = train_rmse.mean(axis=1)
train_std = train_rmse.std(axis=1)
test_mean = test_rmse.mean(axis=1)
test_std = test_rmse.std(axis=1)

fig, ax = plt.subplots(figsize=(10, 6))

ax.fill_between(train_sizes_abs, train_mean - train_std, train_mean + train_std, 
                alpha=0.2, color='#3498db')
ax.fill_between(train_sizes_abs, test_mean - test_std, test_mean + test_std, 
                alpha=0.2, color='#e74c3c')

ax.plot(train_sizes_abs, train_mean, 'o-', color='#3498db', linewidth=2, 
        label='Training Score')
ax.plot(train_sizes_abs, test_mean, 'o-', color='#e74c3c', linewidth=2, 
        label='Validation Score')

ax.set_xlabel('Training Set Size', fontsize=11)
ax.set_ylabel('RMSE (log scale)', fontsize=11)
ax.set_title(f'{best_model_name}: Learning Curve', fontsize=12, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'learning_curve.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"Saved: {FIGURES_DIR / 'learning_curve.png'}")

## 10. Save Best Pipeline & Metrics

In [ ]:
# Save best pipeline
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filename = f"best_pipeline_{timestamp}.joblib"
model_path = MODELS_DIR / model_filename

joblib.dump(best_pipeline, model_path)
print(f"✓ Model saved: {model_path}")

# Save metrics
metrics = {
    'best_model': best_model_name,
    'timestamp': timestamp,
    'dataset': {
        'instances': len(df),
        'features': len(X.columns),
        'numeric_features': get_numeric_features(),
        'categorical_features': get_categorical_features()
    },
    'cv_results': {k: {kk: float(vv) for kk, vv in v.items()} for k, v in results.items()},
    'best_model_metrics': {
        'rmse': float(best_results['rmse_mean']),
        'rmse_std': float(best_results['rmse_std']),
        'mae': float(best_results['mae_mean']),
        'mae_std': float(best_results['mae_std']),
        'r2': float(best_results['r2_mean']),
        'r2_std': float(best_results['r2_std'])
    },
    'model_path': str(model_path)
}

metrics_path = MODELS_DIR / "metrics.json"
with open(metrics_path, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"✓ Metrics saved: {metrics_path}")

In [ ]:
# Final Summary
print("\n" + "=" * 60)
print("TRAINING COMPLETE")
print("=" * 60)
print(f"\n★ Best Model: {best_model_name}")
print(f"  RMSE (CV): {best_results['rmse_mean']:.4f} ± {best_results['rmse_std']:.4f}")
print(f"  MAE (CV):  {best_results['mae_mean']:.4f} ± {best_results['mae_std']:.4f}")
print(f"  R² (CV):   {best_results['r2_mean']:.4f} ± {best_results['r2_std']:.4f}")
print(f"\n📁 Files saved:")
print(f"  Model: {model_path}")
print(f"  Metrics: {metrics_path}")
print(f"  Figures: {FIGURES_DIR}")
print("\n📊 All Model Results:")
for name, res in results.items():
    marker = "★" if name == best_model_name else " "
    print(f"  {marker} {name}: RMSE={res['rmse_mean']:.4f}, R²={res['r2_mean']:.4f}")